# Correlations
**A companion notebook to "Graduate Quantum Information Science"**

In [1]:
import numpy as np
from numpy.linalg import svd 

np.set_printoptions(precision=5)

ModuleNotFoundError: No module named 'numpy'

This notebook is meant to illustrate the concepts from "Graduate Quantum Information Science" concerning measures of two-bit and two-qubit correlations. The approach here mirrors the probability first approach of the "Graduate Quantum Information Science" textbook.

We consider seven examples of two-bit probability distributions, two unentangled states, and five two-qubit entangling gates. All of these combination can be explored using the widgets at the end.

### Table of contents
* [Correlations in probability density vectors](#corr)
    * [Functions for evaluating correlations](#infofuncs)
    * [Examples of correlated probability density vectors](#pdv_examples)
* [Correlations in probability density matrices](#qcorr)
    * [Functions for evaluating quantum correlations](#qinfofuncs)
    * [Examples of unentangled probability density matrices](#pdm_examples)
    * [Examples of entangling operations](#gates)
* [Widgets](#widget)


# Correlations in Probability Density Vectors <a class="anchor" id="corr"></a>

## Functions for evaluating correlations <a class="anchor" id="infofuncs"></a>

In [3]:
#helper functions
from scipy.linalg import logm
def log2m(A):
    """Compute matrix logarithm base 2"""
    return np.log2(np.exp(1)) * logm(A)

#relative entropy
'''def KL_alt(p, q):
    """relative entropy of p wrt q"""
    p = np.matrix(p)
    
  
    a = np.asarray(p, dtype=float)+np.finfo(np.float64).eps
    b = np.asarray(q, dtype=float)+np.finfo(np.float64).eps

    return np.sum(np.where(a != 0, a * np.log2(a / b), 0))
'''

def KL(a, b):
    """relative entropy of a wrt b"""
 
    a = np.matrix(a)
    b = np.matrix(b)

    if min(a.shape)==1 and min(b.shape)==1:
        A = np.diagflat(a)
        B = np.diagflat(b)
    else:
        A = a
        B = b
            
    return np.trace( A @ log2m(A) ) - np.trace( A @ log2m (B) )



def correlations(T):
    """Compute correlations for the coefficents of the two variable probability density vector"""
    V, Sigma, W = svd(T, full_matrices=True)



    pA = [T[0,0] + T[0,1], T[1,0]  + T[1,1]]
    pB = [T[0,0] + T[1,0], T[0,1]  + T[1,1]]

    S_A=0
    for p in pA:
        if abs(p)>1e-9:
            S_A += -p*np.log2(p) 

    S_B=0
    for p in pB:
        if abs(p)>1e-9:
            S_B += -p*np.log2(p)

    S_AB=0
    for l in T.flatten().tolist()[0]:
        if abs(l)>1e-9:
            S_AB+= -l*np.log2(l)

    #correlation measures
    I = S_A + S_B - S_AB
    S_A_given_B = S_AB - S_B
    S_B_given_A = S_AB - S_A

    print("Entropy pA\t\t\t{0:.5f}".format(S_A))
    print("Entropy pB\t\t\t{0:.5f}".format(S_B))
    print("Entropy pAB\t\t\t{0:.5f}".format(S_AB))   
    print("Entropy of A given B\t\t{0:.5f}".format(S_A_given_B))
    print("Entropy of B given A\t\t{0:.5f}".format(S_B_given_A))
    print("")    
    print("Mutual information I(A:B)\t{0:.5f}".format(I))
    print("D(pAB||pA x pB)  \t\t{0:.5f}".format(KL(T.flatten(),np.kron(pA,pB)) ))
    #print("D_2(pAB||pA x pB)  \t\t{0:.5f}".format(KL_alt(T.flatten(),np.kron(pA,pB)) ))
    print("Singular values\t\t\t",Sigma)



# Examples of correlated probability density vectors <a class="anchor" id="pdv_examples"></a>

## Example from "Graduate Quantum Information Science" 

In [4]:
T=np.matrix([[16/40.,  3/40.],  # [Red_Cube Red_Sphere]
             [ 6/40., 15/40.]]) # [BlueCube BlueSphere]  

pA = np.matrix([[T[0,0] + T[0,1]],  # [Red]  19/40
                [T[1,0] + T[1,1]]]) # [Blue] 21/40

pB = np.matrix([T[0,0] 
              + T[1,0],         #  [Cube] (22/40)
                       T[0,1] 
                     + T[1,1]]) #  [Sphere] (18/40)

print("Joint pAB")
print(T)
print("\n")
print("Marginal pA")
print(pA)
print("\n")
print("Marginal pB")
print(pB)
print("\n")
correlations(T)
print("\n\n")

Joint pAB
[[0.4   0.075]
 [0.15  0.375]]


Marginal pA
[[0.475]
 [0.525]]


Marginal pB
[[0.55 0.45]]


Entropy pA			0.99820
Entropy pB			0.99277
Entropy pAB			1.75023
Entropy of A given B		0.75745
Entropy of B given A		0.75203

Mutual information I(A:B)	0.24074
D(pAB||pA x pB)  		0.24074
Singular values			 [0.5025  0.27612]





Note that $p_A$ is slightly closer to uniform than $p_B$. Consequently, the entropy $S(A)$ is slightly greater than $S(B)$.

## No blue cubes

In [5]:
T2=np.matrix([[1/3.,1/3.],  # [Red_Cube Red_Sphere]
              [  0 ,1/3.]]) # [BlueCube BlueSphere] 
print(T2)
correlations(T2)
print("\n\n")

[[0.33333 0.33333]
 [0.      0.33333]]
Entropy pA			0.91830
Entropy pB			0.91830
Entropy pAB			1.58496
Entropy of A given B		0.66667
Entropy of B given A		0.66667

Mutual information I(A:B)	0.25163
D(pAB||pA x pB)  		0.25163
Singular values			 [0.53934 0.20601]





/opt/.qbraid/environments/qbraid_000000/pyenv/lib/python3.11/site-packages/scipy/linalg/_matfuncs.py:204: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  F = scipy.linalg._matfuncs_inv_ssq._logm(A)


Note 1) that the entropy of the marginal is equal to the binary entropy of a bais coin with a 1/3 to 2/3 bias and 2) the entropy of the join distribution is log $3$  since there are 3 equally probable outcomes. Observe:

In [6]:
S=-(2/3)*np.log2(2/3)-(1/3)*np.log2(1/3)
print("entropy with 1/3 bias:\t",S)

print("log(W) for W=3:\t\t",np.log2(3))

entropy with 1/3 bias:	 0.9182958340544896
log(W) for W=3:		 1.584962500721156


## Uniform distribution

In [7]:
T3=np.matrix([[1/4.,1/4.],[1/4.,1/4.]])
print(T3)
correlations(T3)
print("\n\n")

[[0.25 0.25]
 [0.25 0.25]]
Entropy pA			1.00000
Entropy pB			1.00000
Entropy pAB			2.00000
Entropy of A given B		1.00000
Entropy of B given A		1.00000

Mutual information I(A:B)	0.00000
D(pAB||pA x pB)  		0.00000
Singular values			 [5.00000e-01 8.38676e-18]





Notice that the two properties of color and shape are independent in this case. Thus, the entropy of the joint system is the sum of the entropies of the individual systems. The zero mutual information also says that B contains no information about A.

Also notice that the singular values indicate that one of the basis states can be removed without lost. 

## Half red cubes and half blue spheres

In [8]:
T4=np.matrix([[1/2.,0],[0,1/2.]])
print(T4)
correlations(T4)
print("\n\n")

[[0.5 0. ]
 [0.  0.5]]
Entropy pA			1.00000
Entropy pB			1.00000
Entropy pAB			1.00000
Entropy of A given B		0.00000
Entropy of B given A		0.00000

Mutual information I(A:B)	1.00000
D(pAB||pA x pB)  		1.00000
Singular values			 [0.5 0.5]





Notice, in this case, there are only two types of items. If the item is blue then it is a sphere and if it is a cube it is red. Hence, S(A) = S(B) = S(AB). But then there is no conditional entropy: if you are given A then you know B exactly. 

If you observe A then you gain all the information about B. This is reflected in the mutual information value equal to one bit.

## All red cubes

In [9]:
T5=np.matrix([[1,0],[0,0]])
print(T5)
correlations(T5)
print("\n\n")

[[1 0]
 [0 0]]
Entropy pA			0.00000
Entropy pB			0.00000
Entropy pAB			0.00000
Entropy of A given B		0.00000
Entropy of B given A		0.00000

Mutual information I(A:B)	0.00000
D(pAB||pA x pB)  		0.00000
Singular values			 [1. 0.]





This is the trivial case. The entropies are all zero, and one of the singular values is as well.

## All cubes; half red and half blue

In [10]:
T6=np.matrix([[1/2.,0],[1/2.,0]])
print(T6)
correlations(T6)
print("\n\n")

[[0.5 0. ]
 [0.5 0. ]]
Entropy pA			1.00000
Entropy pB			0.00000
Entropy pAB			1.00000
Entropy of A given B		1.00000
Entropy of B given A		0.00000

Mutual information I(A:B)	0.00000
D(pAB||pA x pB)  		0.00000
Singular values			 [0.70711 0.     ]





The entropy of A given B is the full entropy of $p_{AB}$. This is because B contains no information since there are no spheres. Note the separability implied by the zero singular value.

## All red; half spheres, half cubes

In [11]:
T7=np.matrix([[1/2.,1/2.],[0,0]])
print(T7)
correlations(T7)
print("\n\n")

[[0.5 0.5]
 [0.  0. ]]
Entropy pA			0.00000
Entropy pB			1.00000
Entropy pAB			1.00000
Entropy of A given B		0.00000
Entropy of B given A		1.00000

Mutual information I(A:B)	0.00000
D(pAB||pA x pB)  		0.00000
Singular values			 [0.70711 0.     ]





The entropy of B given A is the full entropy of $p_{AB}$. This is because A contains no information since there is no blue. Note the separability implied by the zero singular value.

# Correlations in Quantum Probability Density Matrices <a class="anchor" id="qcorr"></a>

## Functions for evaluating quantum correlations <a class="anchor" id="qinfofuncs"></a>

In [12]:
def partial_trace(r12,index_to_remove):
    """performs the partial trace of rho12 over subspace 1 or 2.
    
    Inputs: 
        rho12             density matrix
        index_to_remove   either zero or one
    """
    
    r0 = [ [ r12[0,0] + r12[1,1], r12[0,2] + r12[1,3] ] , [ r12[2,0] + r12[3,1], r12[2,2] + r12[3,3] ]]
    r1 = [ [ r12[0,0] + r12[2,2], r12[0,1] + r12[2,3] ] , [ r12[1,0] + r12[3,2], r12[1,1] + r12[3,3] ]]
    
    if index_to_remove == 0:
        return r1
    else:
        return r0
    
def partial_trace2(r12,index_to_remove):
    """performs the partial trace of rho12 over subspace 1 or 2.
    
    Inputs: 
        rho12             density matrix
        index_to_remove   either zero or one
    """
    X = np.matrix([[0,1],[1,0]])
    Y = np.matrix([[0,1j],[-1j,0]])
    Z = np.matrix([[1,0],[0,-1]])
    I = np.eye(2)
    
    if index_to_remove == 0:
        A = np.kron(I,X)
        B = np.kron(I,Y)
        C = np.kron(I,Z)
    else:
        A = np.kron(X,I)
        B = np.kron(Y,I)
        C = np.kron(Z,I)
    
    #get bias vector
    r = [np.trace(r12@A), np.trace(r12@B) , np.trace(r12@C)]
        
    return np.matrix( .5* (np.eye(2) +sum(i[0] * i[1] for i in zip(r,[X,Y,Z]))))


def correlations_qm(rhoAB,verbose=False):
    """Compute correlations for the coefficents of the two variable probability density vector"""
    
    rhoA = np.matrix(partial_trace(rhoAB,1))

    rhoB = np.matrix(partial_trace(rhoAB,0))
    
    if(verbose):
        print("rhoA")
        print(rhoA)
        print("rhoB")
        print(rhoB)
    
    S_A = -1 * np.trace(rhoA @ log2m(rhoA))
    S_B = -1 * np.trace(rhoB @ log2m(rhoB))
    
    S_AB= -1 * np.trace(rhoAB @ log2m(rhoAB))

    #correlation measures
    I = S_A + S_B - S_AB
    S_A_given_B = S_AB - S_B
    S_B_given_A = S_AB - S_A

    print("Entropy rhoA\t\t\t{0:.5f}".format(S_A))
    print("Entropy rhoB\t\t\t{0:.5f}".format(S_B))
    print("Entropy rhoAB\t\t\t{0:.5f}".format(S_AB))   
    print("Entropy of A given B\t\t{0:.5f}".format(S_A_given_B))
    print("Entropy of B given A\t\t{0:.5f}".format(S_B_given_A))
    print("")    
    print("Mutual information I(A:B)\t{0:.5f}".format(I))
    print("D(rhoAB||rhoA (x) rhoB)\t\t{0:.5f}".format(KL(rhoAB,np.kron(rhoA,rhoB))))



**Advanced exercise:** Provide a computation of the singular values for $\rho_{AB}$ in `correlations_qm`. You should check that your answers align for lifted probability density vectors using the `correlations` code.

## Examples of unentangled probability density matrices <a class="anchor" id="pdm_examples"></a>

In the next cell, we create two ansatz. In both cases, we will create an unentangled state with the same marginals as in the textbook example. In the first of these ansatz, the local qubit states are pure and line within the $x=y$ plane and have $z$ value that matches the given bias. In the second ansatz, the local qubit states are mixed with non-zero $z$ component matching the given bias.

In [13]:
#
#
#Pure state decorrelated probability density matrices
#
#

T=np.matrix([[16/40.,3/40.],[6/40.,15/40.]])
print("joint pdv")
print("pAB=",T.flatten())
print()


#marginals
pA = np.matrix([T[0,0] + T[0,1], T[1,0]  + T[1,1]])
pB = np.matrix([T[0,0] + T[1,0], T[0,1]  + T[1,1]])
print("marginal pdv's")
print("pA=",pA)
print("pB=",pB)

#bais
bA= pA[0,0]-.5
bB= pB[0,0]-.5

#qubit bais parameters (pure states lying in the x=y plane)
s=np.sqrt(.5*(1-(bA*bA)))
rA= [s,s,bA]

t=np.sqrt(.5*(1-(bB*bB)))
rB= [t,t,bB]

X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,1j],[-1j,0]])
Z = np.matrix([[1,0],[0,-1]])

rhoA = .5* (np.eye(2) +sum(i[0] * i[1] for i in zip(rA,[X,Y,Z])))
rhoB = .5* (np.eye(2) +sum(i[0] * i[1] for i in zip(rB,[X,Y,Z])))

print("")
print("marginal pdm")
print("rhoA=\n",rhoA)
print("rhoB=\n",rhoB)

print("")
print("rhoA \otimes rhoB=")
print(np.kron(rhoA,rhoB))
rho12_pure=np.kron(rhoA,rhoB)

#the marginals should the same as the original
assert(np.allclose(np.matrix(partial_trace(rho12_pure,1)),rhoA))
assert(np.allclose(np.matrix(partial_trace(rho12_pure,0)),rhoB))

joint pdv
pAB= [[0.4   0.075 0.15  0.375]]

marginal pdv's
pA= [[0.475 0.525]]
pB= [[0.55 0.45]]

marginal pdm
rhoA=
 [[0.4875 +0.j      0.35344+0.35344j]
 [0.35344-0.35344j 0.5125 +0.j     ]]
rhoB=
 [[0.525  +0.j      0.35311+0.35311j]
 [0.35311-0.35311j 0.475  +0.j     ]]

rhoA \otimes rhoB=
[[ 2.55938e-01+0.00000e+00j  1.72142e-01+1.72142e-01j
   1.85558e-01+1.85558e-01j -3.52382e-19+2.49609e-01j]
 [ 1.72142e-01-1.72142e-01j  2.31563e-01+0.00000e+00j
   2.49609e-01+3.52382e-19j  1.67885e-01+1.67885e-01j]
 [ 1.85558e-01-1.85558e-01j  2.49609e-01-3.52382e-19j
   2.69062e-01+0.00000e+00j  1.80969e-01+1.80969e-01j]
 [-3.52382e-19-2.49609e-01j  1.67885e-01-1.67885e-01j
   1.80969e-01-1.80969e-01j  2.43437e-01+0.00000e+00j]]


In [14]:
#
#
#Mixed state decorrelated probability density matrices
#
#

T=np.matrix([[16/40.,3/40.],[6/40.,15/40.]])
print("joint pdv")
print("pAB=",T.flatten())

#marginals
print("marginal pdv")
print(pA)
print(pB)

#bais
bA= pA[0,0]-.5
bB= pB[0,0]-.5

#qubit bais parameters (mixed state, z-axis)
rA= [0,0,bA]
rB= [0,0,bB]

X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,1j],[-1j,0]])
Z = np.matrix([[1,0],[0,-1]])

rhoA = np.matrix(.5* (np.eye(2) +sum( i[0] * i[1] for i in zip(rA,[X,Y,Z]))))
rhoB = np.matrix(.5* (np.eye(2) +sum( i[0] * i[1] for i in zip(rB,[X,Y,Z]))))

print("")
print("marginal pdm")
print(rhoA)
print(rhoB)


print("")
print("rhoA \otimes rhoB=")
print(np.kron(rhoA,rhoB))

rho12_mixed = np.kron(rhoA,rhoB)

#the marginals should the same as the original
assert(np.allclose(np.matrix(partial_trace(rho12_mixed,1)),rhoA))
assert(np.allclose(np.matrix(partial_trace(rho12_mixed,0)),rhoB))


joint pdv
pAB= [[0.4   0.075 0.15  0.375]]
marginal pdv
[[0.475 0.525]]
[[0.55 0.45]]

marginal pdm
[[0.4875+0.j 0.    +0.j]
 [0.    +0.j 0.5125+0.j]]
[[0.525+0.j 0.   +0.j]
 [0.   +0.j 0.475+0.j]]

rhoA \otimes rhoB=
[[0.25594+0.j 0.     +0.j 0.     +0.j 0.     +0.j]
 [0.     +0.j 0.23156+0.j 0.     +0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.26906+0.j 0.     +0.j]
 [0.     +0.j 0.     +0.j 0.     +0.j 0.24344+0.j]]


## Examples of entangling gates <a class="anchor" id="gates"></a>

We can mix the two states with different entangling unitary gates given in matrix form.

1. Identity transform

In [15]:
#identity
U12_id = np.matrix([[1, 0, 0, 0] , [ 0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
print(U12_id)

[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


2. Spin coupling

In [16]:
#singlet-triplet
s=1/np.sqrt(2)
U12_spin = np.matrix([[1, 0, 0, 0] , [ 0, s, s, 0], [0, s,-s, 0], [0, 0, 0, 1]])

3. CNOT gate

In [17]:
#CNOT
U12_cnot = np.matrix([[1, 0, 0, 0] , [ 0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

4. CZ (controlled-Z) gate

In [18]:
#CZ
U12_cz = np.matrix([[1, 0, 0, 0] , [ 0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0,-1]])

5. Bell state transformation

In [19]:
#Bell
s=1/np.sqrt(2)
U12_bell = s* np.matrix([[1, 0, 0, 1] , [ 0, 1, 1, 0], [0, 1,-1, 0], [1, 0, 0,-1]])

**Exercise:** Try implementing $U_{12}$ equal to a Haar random 4x4 unitary.  
**Exercise:** Try using $U_{12}=R_n(\theta) \otimes R_m(\phi)$

# Widgets <a class="anchor" id="widget"><a/>
We summarize our findings and examples using two widget that allow you start from either the unentangled ansatze or from the lifted probability density vectors.

In [20]:
#!pip install ipywidgets
#!conda install ipywidgets

## Correlation widget for unentangled ansatze

In [21]:
from ipywidgets import widgets

ModuleNotFoundError: No module named 'ipywidgets'

In [1]:
3+3

6

In [22]:
import ipywidgets as widgets
from IPython.display import clear_output

#print(widgets.Dropdown.__doc__)

w1=widgets.Dropdown(
    options=[('Pure', 'rho12_pure'), ('Mixed', 'rho12_mixed')],
    label='Pure',
    description='PDM ansatz:',
)


w2=widgets.Dropdown(
    options=[('Id', 'U12_id'), ('Singlet-triplet', 'U12_spin'), ('CNOT', 'U12_cnot'),('CZ','U12_cz'),('Bell','U12_bell')],
    label='CNOT',
    description='Unitary 12:',
)


w3 = widgets.Button(
    description='Run',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run'
)

display(w1)
display(w2)
display(w3)

def run_on_click(self, verbose=False):
    
    
    
    #pull ansatz from widget
    r = eval(w1.value)
    
    #pull unitary from widget
    U = eval(w2.value) 
    
    #make rho
    rho = U @ r @ U.H
    
    #outputs
    clear_output()
    
    display(w1)
    display(w2)
    display(w3)
    print()
    print("OUTPUT:")
    print()
    if(verbose):
        print("Selected ansatz")
        print(r)
        print("Selected unitary")
        print(U)
    
    print("rhoAB")
    print(rho)
    correlations_qm(rho,True)
    print()
              
    

w3.on_click(run_on_click)




ModuleNotFoundError: No module named 'ipywidgets'

Note that the conditional entropy often becomes negative when the total state is pure. The conditional entropy is given by $S(A|B) = S(AB) - S(A)$, so when $S(AB)=0$ negative values can result. This was discussed in the chapter and discussed further in Ref. https://arxiv.org/abs/quant-ph/9512022.

## Correlation widget for the lifted probability density vectors

In [40]:
# example from text    
T = np.matrix([[16/40.,3/40.],[6/40.,15/40.]])
R = np.diagflat(T.flatten())

# no blue cubes
T2=np.matrix([[1/3.,1/3.],[0,1/3.]])
R2 = np.diagflat(T2.flatten())

# uniform
T3=np.matrix([[1/4.,1/4.],[1/4.,1/4.]])
R3 = np.diagflat(T3.flatten())

# Half red cubes and half blue spheres 
T4=np.matrix([[1/2.,0],[0,1/2.]])
R4 = np.diagflat(T4.flatten())

# All red cubes
T5=np.matrix([[1,0],[0,0]])
R5 = np.diagflat(T5.flatten())

# All cubes; half red and half blue
T6=np.matrix([[1/2.,0],[1/2.,0]])
R6 = np.diagflat(T6.flatten())

# All red; half spheres, half cubes
T7=np.matrix([[1/2.,1/2.],[0,0]])
R7 = np.diagflat(T7.flatten())


In [41]:
import ipywidgets as widgets
from IPython.display import clear_output

#print(widgets.Dropdown.__doc__)

w4=widgets.Dropdown(
    options=[('From text', 'R'), ('No blue cubes', 'R2'), ('Uniform', 'R3'), ( 'Half red cubes and half blue spheres', 'R4') , ('Only red cubes', 'R5'), ('All cubes','R6'), ('All red','R7')],
    label='Uniform',
    description='PDV:',
)


w5=widgets.Dropdown(
    options=[('Id', 'U12_id'), ('Singlet-triplet', 'U12_spin'), ('CNOT', 'U12_cnot'),('CZ','U12_cz'),('Bell','U12_bell')],
    label='Id',
    description='Unitary 12:',
)


w6 = widgets.Button(
    description='Run',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run'
)

display(w4)
display(w5)
display(w6)

def run_on_click_6(self,verbose=True):
    
    
    #pull lifted pdv from widget
    r = eval(w4.value)
    
    #pull unitary from widget
    U = eval(w5.value) 
     
    

    
    rho = U @ r @ U.H
    
    #outputs
    clear_output()

    display(w4)
    display(w5)
    display(w6)
    print()
    print("OUTPUT:")
    print()
    if(verbose):
        print("Selected PDV")
        print(r)
        print("Selected unitary")
        print(U)
    
    print("rhoAB")
    print(rho)
    correlations_qm(rho,True)
    print()
    
w6.on_click(run_on_click_6)




Dropdown(description='PDV:', index=2, options=(('From text', 'R'), ('No blue cubes', 'R2'), ('Uniform', 'R3'),…

Dropdown(description='Unitary 12:', options=(('Id', 'U12_id'), ('Singlet-triplet', 'U12_spin'), ('CNOT', 'U12_…

Button(description='Run', style=ButtonStyle(), tooltip='Run')

Note when $U_{12}=\mathbf{1}$, we get the standard information theoretic values as computed in the first section.

In [37]:
# JDWhitfield 2024